In [89]:
import cv2
import pickle
import cvzone
import numpy as np

In [90]:
# Video feed
cap = cv2.VideoCapture('carPark.mp4')

In [91]:
with open('CarParkPos', 'rb') as f:
    posList = pickle.load(f)

In [92]:
width, height = 107, 48

In [ ]:
def checkParkingSpace(imgPro):
    spaceCounter = 0

    for pos in posList:
        x, y = pos

        imgCrop = imgPro[y:y + height, x:x + width]
        count = cv2.countNonZero(imgCrop)

        # Threshhold value, adjusted based on video
        if count < 900:
            color = (0, 255, 0)
            thickness = 5
            spaceCounter += 1
        else:
            color = (0, 0, 255)
            thickness = 2

        cv2.rectangle(img, pos, (pos[0] + width, pos[1] + height), color, thickness)
        cvzone.putTextRect(img, str(count), (x, y + height - 3), scale=1,
                           thickness=2, offset=0, colorR=color)

    cvzone.putTextRect(img, f'Free: {spaceCounter}/{len(posList)}', (100, 50), scale=3,
                           thickness=5, offset=20, colorR=(0,200,0))

# Main loop
while True:
    # Close video when finished
    if cap.get(cv2.CAP_PROP_POS_FRAMES) == cap.get(cv2.CAP_PROP_FRAME_COUNT):
        break

    success, img = cap.read()
    # Preprocessing
    imgGray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    imgBlur = cv2.GaussianBlur(imgGray, (3, 3), 1)      # Kernel size (3,3) and sigma 1
    # Adaptive thresholding, block size 25, C 16
    imgThreshold = cv2.adaptiveThreshold(imgBlur, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                                         cv2.THRESH_BINARY_INV, 25, 16)     
    imgMedian = cv2.medianBlur(imgThreshold, 5)
    kernel = np.ones((3, 3), np.uint8)
    imgDilate = cv2.dilate(imgMedian, kernel, iterations=1)

    # Check spaces
    checkParkingSpace(imgDilate)

    cv2.imshow("Image", img)
    # cv2.imshow("Processed", imgDilate)
    cv2.waitKey(5)
    if cv2.waitKey(10) & 0xFF == 27:  # ESC to quit
        break

cap.release()

In [94]:
cv2.destroyAllWindows()